In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import DarkNews as dn

from DarkNews import const

from DarkNews.GenLauncher import GenLauncher

from DarkNews import Cfourvec as Cfv

import particle.literals as lp

In [3]:
MODEL_KWARGS = {'HNLtype': 'dirac', 'UD4': 1.0, 'alphaD': 0.25, 'Umu4': np.sqrt(9e-7), 'epsilon': np.sqrt(2e-10/const.alphaQED)}

gen = GenLauncher(mzprime=0.03, m4=0.420, neval=1000, exp="miniboone_fhc", loglevel='INFO', seed=42, **MODEL_KWARGS)
df=gen.run(loglevel="INFO")

print(df.w_event_rate.sum())

---------------------------------------------------------
   ______           _        _   _                     
   |  _  \         | |      | \ | |                    
   | | | |__ _ _ __| | __   |  \| | _____      _____   
   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  
   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  
   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  

---------------------------------------------------------
Model:
	1 dirac heavy neutrino(s).

---------------------------------------------------------
Experiment:
	MiniBooNE_FHC
	fluxfile loaded: ../fluxes/MiniBooNE_FHC.dat
	POT: 1.875e+21
	nuclear targets: ['C12', 'H1']
	fiducial mass: [701.1428571428571, 116.85714285714286] tonnes

Directory tree for this run already exists. Overriding it.
---------------------------------------------------------
Generating Events using the neutrino-nucleus upscattering engine

nu(mu) C12 --> N4  C12 --> nu_light e+ e- C12
Helicity conserving upscattering.
N4 de

In [6]:
import vegas 
import gvar
gvar.__version__

'11.10'

In [14]:
from DarkNews import fourvec as fv

In [14]:
gen = GenLauncher(mzprime=0.03, m4=0.420, epsilon=epsilon_def, Umu4=umu4_def, UD4=ud4_def, gD=gD_def, 
                    neval=1000, HNLtype="dirac", exp="miniboone_fhc", loglevel='INFO',
                    seed=333,
                    parquet=True, numpy=True, hepevt=True, sparse=True, print_to_float32=True)

df=gen.run(loglevel="ERROR")
df_2=gen.run(loglevel="ERROR")

Initializing the three-portal model.
---------------------------------------------------------
   ______           _        _   _                     
   |  _  \         | |      | \ | |                    
   | | | |__ _ _ __| | __   |  \| | _____      _____   
   | | | / _  | ___| |/ /   | .   |/ _ \ \ /\ / / __|  
   | |/ / (_| | |  |   <    | |\  |  __/\ V  V /\__ \  
   |___/ \__,_|_|  |_|\_\   \_| \_/\___| \_/\_/ |___/  

---------------------------------------------------------
Model:
	1 dirac heavy neutrino(s).

---------------------------------------------------------
Experiment:
	MiniBooNE_FHC
	fluxfile loaded: ../fluxes/MiniBooNE_FHC.dat
	POT: 1.875e+21
	nuclear targets: ['C12', 'H1']
	fiducial mass: [701.1428571428571, 116.85714285714286] tonnes

Directory tree for this run already exists. Overriding it.


In [ ]:
new_c(df)

In [ ]:
%prun 

gen.dn_printer.print_events_to_HEPEVT()

In [ ]:
filename="./data/miniboone_fhc/3plus1/m4_0.42_mzprime_0.03_dirac/"

df_std = pd.read_pickle(f"{filename}pandas_df.pckl")
df_pq = pd.read_parquet(f"{filename}pandas_df.parquet", engine='pyarrow')
nda = np.load(f"{filename}ndarray.npy")
# ndz = np.load(f"{filename}ndarray.npz")


In [ ]:
nda_fix=np.reshape(nda,(np.shape(nda)[0],int(np.shape(nda)[1]/4),4)).T



### npz format tests

In [ ]:
cols = [f'{v[0]}_{v[1]}' if v[1] else f'{v[0]}' for v in df.columns.values]
print(cols)

In [ ]:
# test that numpy array and dataframe formats save the same information
assert (df_std.to_numpy()[nda!=0]/nda[nda!=0]!=1).sum() == 0 
assert (df_pq.to_numpy()[nda!=0]/nda[nda!=0]!=1).sum() == 0

In [ ]:
if 'as':
    print("Error")

# Vegas tests

In [ ]:
# Testing the VEGAS integration methods:
import vegas as vg
from collections import OrderedDict

class test_integral(vg.BatchIntegrand):

    def __init__(self, dim, c = 1):
        self.dim = dim	
        self.c = c
        self.analytical_func = lambda x: np.exp(self.c * x) * (self.c*x-1)/self.c**2
        self.analytical_res  = (self.analytical_func(2) - self.analytical_func(0))

        # initialize
        self.norm = {'diff_xsec': 1, 'diff_xsec2': 1}
        # normalize integrand with an initial throw
        _throw = self.__call__(np.random.rand(dim,500), np.ones((dim,500)))
        for key,val in _throw.items():
            self.norm[key] = np.mean(val)
    
    def __call__(self, x, jac):

        xmax = 2
        xmin = 0
        xx=np.empty(0)
        diff = 1
        diff2 = 1
        for d in range(self.dim):
            xx = x[:,d]*(xmax - xmin) + xmin
            diff *= np.exp(self.c*xx)*xx
            if d < self.dim - 2:
                diff2 *= np.exp(self.c*xx)*xx

        # hypercube jacobian (vegas hypercube --> physical limits) transformation
        hypercube_jacobian = (xmax - xmin)
        diff  *= hypercube_jacobian**self.dim
        diff2 *= hypercube_jacobian**(self.dim)

        ##############################################
        # return all differential quantities of interest
        self.int_dic = OrderedDict()		
        self.int_dic['diff_xsec'] = diff
        self.int_dic['diff_xsec2'] = diff2
        
        ##############################################
        # storing normalization for integrands to be of O(1) numbers		
        # normalization
        for key in self.int_dic:
            self.int_dic[key] /= self.norm[key]
        self.int_dic['diff_xsec2'] /= jac[:,-1]*jac[:,-2]
        return self.int_dic

In [ ]:
evals = []
evals2 = []
for i in range(5):
    #########################################################################
    # BATCH SAMPLE INTEGRAND OF INTEREST
    DIM = 6
    batch_f = test_integral(dim=DIM)
    integ = vg.Integrator(DIM*[[0.0, 1.0]]) # unit hypercube
    result = dn.MC.run_vegas(batch_f, integ, NINT=20, NEVAL=10000, NINT_warmup=20, NEVAL_warmup=1000)
    ##########################################################################
    # print(result.summary())
    evals.append(result['diff_xsec'].mean*batch_f.norm['diff_xsec'])
    evals2.append(result['diff_xsec2'].mean*batch_f.norm['diff_xsec2'])

plt.plot(evals/evals[0], color='blue')
plt.plot(evals/batch_f.analytical_res**DIM, ls='--', color='blue')
plt.plot(evals2/evals2[0], color='orange')
plt.plot(evals2/batch_f.analytical_res**(DIM-2)/4, ls='--', color='orange')


In [ ]:
samples, weights, jac = dn.MC.get_samples(integ, batch_f, return_jac=True)

In [ ]:
np.sum(weights['diff_xsec'])*batch_f.norm['diff_xsec']/batch_f.analytical_res**DIM


In [ ]:
np.sum(weights['diff_xsec'])*batch_f.norm['diff_xsec']/batch_f.analytical_res**(DIM)


In [ ]:
for key,vals in weights.items():
    plt.plot(vals)

In [ ]:
(weights['diff_xsec2']*jac[:,5]).sum()


In [ ]:
bins = 20

_ = plt.hist(samples[0,:]*2, weights=weights['diff_xsec2']*batch_f.norm['diff_xsec2']/(2/bins), bins=bins, density=False)

x=np.linspace(0,2)

plt.plot(x, np.exp(batch_f.c * x)*x * batch_f.analytical_res**(DIM-3)*4)


# profile amplitude calculation

In [ ]:
proton = dn.detector.NuclearTarget("H1")
bsm_model = dn.model.create_3portal_HNL_model(mzprime=0.1, m4 = 0.01, Umu4=1e-3)
calculator = dn.MC.XsecCalc(bsm_model = bsm_model, scattering_regime = 'p-el', nuclear_target= proton, helicity = 'conserving')

In [ ]:
one = np.ones(1000)
%prun dn.amplitudes.upscattering_dxsec_dQ2([one,one,one], calculator.ups_case)

# profile full generation

In [ ]:
ud4_def = 1.0
alphaD = 0.25
gD_def = np.sqrt(alphaD*4*np.pi)
umu4_def = np.sqrt(9e-7)
ud4 = 1.
epsilon_def = np.sqrt(2e-10/const.alphaQED)

gen = GenLauncher(mzprime=0.03, m4=0.420, epsilon=epsilon_def, Umu4=umu4_def, UD4=ud4_def, gD=gD_def, neval=1000, HNLtype="dirac", exp="miniboone_fhc", loglevel='INFO')
gen.run(log="INFO")
df_mini = gen.df

In [ ]:
%prun gen.run(log="ERROR")

In [ ]:
df_mini.w_event_rate.sum()*0.05


In [ ]:
gen = GenLauncher(mzprime=0.03, m4=0.420, epsilon=epsilon_def, Umu4=umu4_def, UD4=ud4_def, gD=gD_def, neval=1000, HNLtype="dirac", exp="microboone", loglevel='ERROR')
gen.run(log="ERROR")
df_micro = gen.df

In [ ]:
ud4_def = 1.0
alphaD = 0.4
gD_def = np.sqrt(alphaD*4*np.pi)
umu4_def = 4e-4
umu5_def = 4e-4
ud4 = 1.
ud5 = 1.
epsilon_def = 2.2e-2

gen = GenLauncher(mzprime=1.25, m4=0.010, m5=0.490, epsilon=epsilon_def, Umu4=umu4_def, Umu5=umu5_def, UD4=ud4_def, UD5=ud5, gD=gD_def, neval=10000, HNLtype="majorana", exp="miniboone_fhc", loglevel='ERROR')
gen.run(log="ERROR")
df_mini = gen.df

gen = GenLauncher(mzprime=1.25, m4=0.010, m5=0.490, epsilon=epsilon_def, Umu4=umu4_def, Umu5=umu5_def, UD4=ud4_def, UD5=ud5, gD=gD_def, neval=10000, HNLtype="majorana", exp="microboone", loglevel='ERROR')
gen.run(log="ERROR")
df_micro = gen.df

In [ ]:

ud4_def = 1.0
alphaD = 0.4
gD_def = np.sqrt(alphaD*4*np.pi)
umu4_def = 4e-4
umu5_def = 4e-4
ud4 = 1.
ud5 = 1.
epsilon_def = 2.2e-2

gen = GenLauncher(m4=0.10, epsilon=0.0, dmu4= 1e-6, gD=0.0, neval=1000, HNLtype="dirac", exp="miniboone_fhc", loglevel='ERROR', decay_product='photon')
gen.run(log="ERROR")
df_mini = gen.df


In [ ]:
df_mini['P_decay_N_parent'][['1','2','3']].iloc[0,:]

In [ ]:
print(f" {dn.printer.print_in_order(df_mini.iat[0,'P_projectile'])}")

In [ ]:
%%prun -l 5

gen3p1 = GenLauncher(hepevt=False, m4=0.420, epsilon=0.0, mu_tr_mu4= 1e-6, nopelastic=False, gD=0.0, neval=10000, HNLtype="dirac", exp="miniboone_fhc", loglevel='ERROR', decay_products='photon')
# gen3p2 = GenLauncher(hepevt=False, m4=0.420, epsilon=0.0, mu_tr_mu4= 1e-6, nopelastic=False, gD=0.0, neval=10000, HNLtype="majorana", exp="miniboone_fhc", loglevel='ERROR', decay_products='photon')
gen3p2 = GenLauncher(hepevt=False, m4=0.370, m5=0.420, epsilon=0.0, mu_tr_mu5= 1e-6, mu_tr_45= 1e-4, nopelastic=False, gD=0.0, neval=10000, HNLtype="dirac", exp="miniboone_fhc", loglevel='ERROR', decay_products='photon')
gen3p1.run(log="ERROR")
gen3p2.run(log="ERROR")
df_3p1 = gen3p1.df
df_3p2 = gen3p2.df

In [ ]:
# energy

bins = np.linspace(0,2,20)
df = df_3p1
mask = (df.scattering_regime=='p-el')
x=df.P_decay_gamma.to_numpy()[:,0]
w=df.w_event_rate.to_numpy()
_ = plt.hist(x, weights=w, bins=bins, histtype='step', color='dodgerblue', ls='-')
_ = plt.hist(x[mask], weights=w[mask], bins=bins, histtype='step', color='orange', ls='-')

df = df_3p2
mask = (df.scattering_regime=='p-el')
x=df.P_decay_gamma.to_numpy()[:,0]
w=df.w_event_rate.to_numpy()
_ = plt.hist(x, weights=w, bins=bins, histtype='step', color='dodgerblue', ls='--')
_ = plt.hist(x[mask], weights=w[mask], bins=bins, histtype='step', color='orange', ls='--')

In [ ]:
# angle

bins = np.linspace(-1,1,20)
df = df_3p1
mask = (df.scattering_regime=='p-el')
x=dn.Cfv.get_cosTheta(df.P_decay_gamma.to_numpy())
w=df.w_event_rate.to_numpy()
_ = plt.hist(x, weights=w, bins=bins, histtype='step',color='dodgerblue', ls='-')
_ = plt.hist(x[mask], weights=w[mask], histtype='step',bins=bins, color='orange', ls='-')

df = df_3p2
mask = (df.scattering_regime=='p-el')
x=dn.Cfv.get_cosTheta(df.P_decay_gamma.to_numpy())
w=df.w_event_rate.to_numpy()
_ = plt.hist(x, weights=w, bins=bins, histtype='step', color='dodgerblue', ls='--')
_ = plt.hist(x[mask], weights=w[mask], bins=bins, histtype='step', color='orange', ls='--')

In [ ]:
print(f"event rate Mini {df_micro['w_event_rate'].sum()*0.05}")
print(f"event rate Micro {df_micro['w_event_rate'].sum()*0.05*87/818*(550/470)**2*6/18.75}")

In [ ]:
x = Cfv.get_cosTheta((df_mini['P_decay_ell_minus']+df_mini['P_decay_ell_plus']).to_numpy())
w = df_mini['w_event_rate'].to_numpy()
_ = plt.hist(x, weights=w, bins=np.linspace(-1,1,21))

In [ ]:

print(f"event rate Mini {df_mini['w_event_rate'].sum()*0.05}")
print(f"event rate Micro rescaled {df_mini['w_event_rate'].sum()*87/818*(550/470)**2*6/18.75}")
print(f"event rate Micro {df_micro['w_event_rate'].sum()*0.5}")
print(f"event rate SBND {df_micro['w_event_rate'].sum()*112/87*(470/110)**2*0.5}")
print(f"event rate Icarus {df_micro['w_event_rate'].sum()*476/87*(470/600)**2*0.5}")

In [ ]:
from DarkNews import Cfourvec as Cfv

In [ ]:

Cfv.get_cosTheta(df_micro['P_decay_ell_minus'].to_numpy)


In [ ]:
ud4_def = 1.0
alphaD = 0.25
gD_def = np.sqrt(alphaD*4*np.pi)
umu4_def = np.sqrt(9e-8)
ud4 = 1.
epsilon_def = np.sqrt(2e-10/const.alphaQED)

gen = GenLauncher(mzprime=0.03, m4=0.100, epsilon=epsilon_def, Umu4=umu4_def, UD4=ud4_def, gD=gD_def, neval=10000, HNLtype="dirac", exp="miniboone_fhc", loglevel='ERROR')
gen.run(log="ERROR")

In [ ]:
gen_m = GenLauncher(mzprime=0.03, m4=0.100, epsilon=epsilon_def, Umu4=umu4_def, UD4=ud4_def, gD=gD_def, neval=10000, HNLtype="majorana", exp="miniboone_fhc", loglevel='ERROR')
gen_m.run(log="ERROR")

In [ ]:
df = gen.df
df_m = gen_m.df

In [ ]:
print(df['w_event_rate'].sum()*0.047)
print(df_m['w_event_rate'].sum()*0.047)

In [ ]:
df = []
for i in range(0,2):
    gen_object = GenLauncher(m4 = 0.100, mzprime = 0.03, neval = 10000, nitn=20)
    gen_object.run(log="INFO")
    df_1 = gen_object.df

    # gen_object = GenLauncher(m4 = 0.100, mzprime = 0.02, neval = 1000, nint=20)
    gen_object.run(log="INFO")
    df_2 = gen_object.df

    df.append([df_1, df_2])

In [ ]:
import matplotlib.pyplot as plt

ratios_rate = []
ratios_fxsec = []
ratios_decay = []
for pair in df:
    ratios_rate.append(np.sum(pair[0]['w_event_rate'])/np.sum(pair[1]['w_event_rate']))
    ratios_fxsec.append(np.sum(pair[0]['w_flux_avg_xsec'])/np.sum(pair[1]['w_flux_avg_xsec']))
    
    ratios_decay.append(np.sum(pair[0]['w_decay_rate_0'])/np.sum(pair[1]['w_decay_rate_0']))

plt.scatter(range(2),ratios_rate, label="rate")
plt.scatter(range(2),ratios_fxsec, label="fxsec")
plt.scatter(range(2),ratios_decay, label="decay")
plt.legend(frameon=False)

# plt.scatter(range(0,2),case1, c='blue')
# plt.scatter(range(0,2),case2, c='orange')

In [ ]:
plt.plot(df_1['w_flux_avg_xsec'])
# plt.yscale("log")
# plt.ylim(1,1e7)


In [ ]:
df1,df2 = df[0]
p1 = (df1['P_projectile'] - df1['P_decay_N_parent']).to_numpy()
h1 = dn.Cfourvec.dot4(p1,p1)

p2 = (df2['P_projectile'] - df2['P_decay_N_parent']).to_numpy()
h2 = dn.Cfourvec.dot4(p2,p2)

_ = plt.hist(np.sqrt(-h1), bins=100, range=(0,0.5), histtype='step', weights=df1['w_event_rate'], lw=1)
_ = plt.hist(np.sqrt(-h2), bins=100, range=(0,0.5), histtype='step', weights=df2['w_event_rate'], lw=1)


In [ ]:
p1 = (df_2['P_projectile']).to_numpy()
# p1 = (df_1['P_decay_ell_minus']+df_1['P_decay_ell_plus']).to_numpy()
# p1 = (df_1['P_decay_ell_minus']+df_1['P_decay_ell_plus']+df_1['P_decay_N_daughter']).to_numpy()
h1 = p1[:,0]
_ = plt.hist(h1, bins=30, histtype='step', range=(0,2), weights=df_2['w_flux_avg_xsec'],density=True, label='True neutrino energy')
plt.xlabel("Enu")
# plt.yscale("log")


x = np.linspace(0,2, 1000)
y = df_1.attrs['experiment'].FLUX_FUNCTIONS[1](x)/( df_1.attrs['experiment'].FLUX_FUNCTIONS[1](x)*(x[1]-x[0])).sum()

plt.plot(x,y, label='Input flux')
plt.legend()

In [ ]:

p1 = (df_2['P_decay_ell_minus']+df_2['P_decay_ell_plus']).to_numpy()

h1 = p1[:,0]
_ = plt.hist(h1, bins=30, histtype='step', range=(0,2), weights=df_2['w_event_rate'],density=True, label='e+e- energy')
plt.xlabel("e+ e- total energy")
# plt.yscale("log")


x = np.linspace(0,2, 1000)
y = df_1.attrs['experiment'].FLUX_FUNCTIONS[1](x)/( df_1.attrs['experiment'].FLUX_FUNCTIONS[1](x)*(x[1]-x[0])).sum()

plt.plot(x,y, label='Input flux')
plt.legend()

In [ ]:
p1 = (df_1['P_projectile']).to_numpy()
x = p1[:,0]
y  = (df_1['P_decay_N_parent']).to_numpy()[:,2]
_ = plt.scatter(x, y, marker='.', s=3,  label='True neutrino energy')
plt.xlabel("Enu")
